In [ ]:
import datetime

import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
sns.set_theme(style="whitegrid", palette="deep")
task_palette = sns.color_palette("deep")
plt.rcParams["text.usetex"] = True
plt.rcParams.update({"figure.titlesize": "small"})
plt.rcParams.update({"axes.titlesize": "small"})
plt.rcParams.update({"axes.labelsize": "small"})
plt.rcParams.update({"ytick.labelsize": "small"})
plt.rcParams.update({"xtick.labelsize": "small"})
plt.rcParams.update({"legend.fontsize": "small"})

In [ ]:
path = "./results/results-mcs_scale_bfs-2024-05-21_17-33-33.csv"
df = pd.read_csv(path, sep=";")
df

In [ ]:
df["duration_s"] = df["duration_ns"] / 1e9

In [ ]:
df["search_type"] = df["use_idlesim"].apply(lambda x: "ACBFS" if x else "BFS")

In [ ]:
df = df.dropna(how="any")

In [ ]:
s = df.shape[0] * [True]
df_plot = df.loc[s]
df_piv = df_plot.copy().pivot(
    index="taskset_position",
    columns="search_type",
    values=["duration_s", "visited_count", "max_period", "nbt"],
)
df_piv.columns = list(map(lambda x: "_".join(x), df_piv.columns))

In [ ]:
df_piv = df_piv.dropna(how="any")

In [ ]:
df_piv["Number of tasks"] = df_piv["nbt_BFS"].astype(int)
df_piv["Period max"] = df_piv["max_period_ACBFS"].astype(int)
df_piv["n"] = df_piv["nbt_BFS"].astype(int)
df_piv["T max"] = df_piv["max_period_ACBFS"].astype(int)

In [ ]:
f, ax = plt.subplots(figsize=(6, 3))

sns.scatterplot(
    data=df_piv,
    x="duration_s_BFS",
    y="duration_s_ACBFS",
    style="T max",
    hue="n",
    ax=ax,
    palette=task_palette,
)
ax.set(xscale="log", yscale="log")
# ax.axis("equal")

lims = [
    10**-5.1,
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, "k--", alpha=0.75, zorder=1)

ax.set(xlim=lims, ylim=lims)

ax.set_xlabel("BFS")
ax.set_ylabel("ACBFS")
ax.set_title("Execution time (s)")

# sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1.1))

f.suptitle(r"$P_{\mathsf{HI}} = 0.5, T^{\mathsf{min}} = 5, U^{*} = 0.5$", y=1.02)

h, l = ax.get_legend_handles_labels()
ax.legend_.remove()
leg = [
    "$n$",
    "2",
    "3",
    "4",
    "5",
    "6",
    "$T^{\mathsf{max}}$",
    "10",
    "15",
    "20",
    "25",
    "30",
]
f.legend(h, leg, ncol=2, loc="upper left", bbox_to_anchor=(0.15, 0.85), framealpha=1)


x = np.log10(df_piv.sort_values(by="duration_s_BFS")["duration_s_BFS"])
y = np.log10(df_piv.sort_values(by="duration_s_BFS")["duration_s_ACBFS"])
m, b = np.polyfit(x, y, 1)
x_reg = np.linspace(lims, 100)
x_reg = np.log10(x_reg)
plt.plot(10**x_reg, 10 ** (m * x_reg + b), color="k", ls=":")

In [ ]:
f.savefig(
    f"./fig_rtss//{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_bfs_acbfs.pdf",
    bbox_inches="tight",
)
f.savefig(
    "./fig_rtss/bfs_acbfs.pdf",
    bbox_inches="tight",
)

In [ ]:
path = "./results/results-mcs_scale01-2024-05-21_14-37-22.csv"
path = "./results/results-mcs_scale01-2024-05-21_20-34-48.csv"
path = "./results/results-mcs_scale01-2024-05-22_09-18-18.csv"
path = "./results/results-mcs_scale01-2024-05-22_22-32-29.csv"
path = "./results/results-mcs_scale01-2024-05-22_22-32-29 (1).csv"
path = "./results/results-mcs_scale01-2024-05-23_18-03-14.csv"

df_sp1 = pd.read_csv(
    "./results/results-mcs_scale01-2024-05-22_22-32-29 (1).csv", sep=";"
)
df_sp2 = pd.read_csv("./results/results-mcs_scale01-2024-05-23_18-03-14.csv", sep=";")
df_sp = pd.concat([df_sp1, df_sp2], ignore_index=True)
df_sp

In [ ]:
df_sp2

In [ ]:
df_sp["taskset_file"].value_counts()

In [ ]:
df_sp.groupby("taskset_file")["taskset_position"].max()

In [ ]:
df_sp["duration_s"] = df_sp["duration_ns"] / 1e9

In [ ]:
df_sp["timeout_int"] = df_sp["timeout"].astype(int)
df_sp.groupby(["nbt", "max_period"])["timeout_int"].agg(["count", "sum"])

In [ ]:
df_sp.groupby("nbt")["duration_s"].describe()

In [ ]:
# f, axs = plt.subplots(2, 1, figsize=(3.5*52/98*2, 3.5), gridspec_kw={"hspace": 0.25})
f, axs = plt.subplots(
    2, 1, figsize=(3.2 / 1.05, 3.2), gridspec_kw={"hspace": 0.25}, sharey=True
)


ax1 = axs[0]
ax2 = ax1.twinx()

s = df_sp["taskset_file"].str.contains("task")
dimension = "nbt"

# s = df_sp["taskset_file"].str.contains("period")
# dimension = "max_period"

s &= df_sp["use_case"] == "ACBFS, no oracle"


n_xp = 50

x_val = np.sort(df_sp.loc[s, dimension].unique())
x_val = list(map(int, x_val))
y_val_line = []
y_val_bar = []
for x in x_val:
    y_line = df_sp.loc[s & (df_sp[dimension] == x), "duration_s"].mean()
    y_val_line.append(y_line)

    y_bar = df_sp.loc[s & (df_sp[dimension] == x), "timeout_int"].sum()
    y_bar += n_xp - df_sp.loc[s & (df_sp[dimension] == x), "timeout_int"].count()
    y_val_bar.append(y_bar)

sns.lineplot(
    x=x_val,
    y=y_val_line,
    ax=ax1,
    marker="o",
    ms=9,
    estimator="median",
)
ax1.set(yscale="log")

ax2.bar(
    x=x_val,
    height=y_val_bar,
    color="w",
    edgecolor=task_palette[1],
    hatch="//",
    alpha=1,
    width=(x_val[1] - x_val[0]) * 0.8,
)

ax2.grid(False)
ax2.set_ylim(0, n_xp)


ax1.set_xlabel("Number of tasks ($n$), $T^{\mathsf{max}} = 30$")
ax1.set_ylabel("Execution time (s)")
ax2.set_ylabel("Timeouts")
ax1.minorticks_off()

ax1.xaxis.tick_top()
ax1.xaxis.set_label_position("top")
ax1.xaxis.set_ticks_position("none")
ax1.yaxis.set_ticks_position("none")
ax2.yaxis.set_ticks_position("none")
ax1.tick_params(axis="x", which="major", pad=0)


ax1 = axs[1]
ax2 = ax1.twinx()

# s = df_sp["taskset_file"].str.contains("task")
# dimension = "nbt"

s = df_sp["taskset_file"].str.contains("period")
dimension = "max_period"

s &= df_sp["use_case"] == "ACBFS, no oracle"


x_val = np.sort(df_sp.loc[s, dimension].unique())
x_val = list(map(int, x_val))
y_val_line = []
y_val_bar = []
for x in x_val:
    y_line = df_sp.loc[s & (df_sp[dimension] == x), "duration_s"].mean()
    y_val_line.append(y_line)

    y_bar = df_sp.loc[s & (df_sp[dimension] == x), "timeout_int"].sum()
    y_bar += n_xp - df_sp.loc[s & (df_sp[dimension] == x), "timeout_int"].count()
    y_val_bar.append(y_bar)

sns.lineplot(
    x=x_val,
    y=y_val_line,
    ax=ax1,
    marker="o",
    ms=9,
    estimator="median",
)
ax1.set(yscale="log")

ax2.bar(
    x=x_val,
    height=y_val_bar,
    color="w",
    edgecolor=task_palette[1],
    hatch="//",
    alpha=1,
    width=(x_val[1] - x_val[0]) * 0.8,
)

ax2.grid(False)
ax2.set_ylim(0, n_xp)


ax1.set_xlabel("Period max ($T^{\mathsf{max}}$), $n = 5$")
ax1.set_ylabel("Execution time (s)", labelpad=10)
ax2.set_ylabel("Timeouts")
ax1.minorticks_off()
ax1.yaxis.set_ticks_position("none")
ax2.yaxis.set_ticks_position("none")

f.suptitle(
    r"$P_{\mathsf{HI}} = 0.5, T^{\mathsf{min}} = 5, U^{*} = 0.5$",
    # x=0.55,
    # y=0.96,
    y=1.05,
)

blue_line = mlines.Line2D(
    [],
    [],
    color=task_palette[0],
    marker="o",
    linestyle="-",
    markeredgecolor="w",
    label="Execution time (s)",
    ms=9,
)

orange_box = mpatches.Patch(
    hatch="//", label="Timeouts", edgecolor=task_palette[1], facecolor="white"
)

new_handles = [blue_line, orange_box]


ax1.legend(
    handles=new_handles,
    loc="upper center",
    ncol=2,
    framealpha=0,
    columnspacing=0.8,
    bbox_to_anchor=(0.5, 1.28),
)

ax1.get_ylim()


f.tight_layout()

In [ ]:
f.savefig(
    f"./fig_rtss/{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_tmax_ntask.pdf",
    bbox_inches="tight",
)
f.savefig(
    "./fig_rtss/tmax_ntask.pdf",
    bbox_inches="tight",
)

In [ ]:
path = "./results/results-mcs_schedulability-2024-05-18_16-49-40.csv"
df_sch = pd.read_csv(path, sep=";")
df_sch

In [ ]:
df_edfvd_test = df_sch[["taskset_position", "EDFVD_test", "U"]]
df_edfvd_test = df_edfvd_test.rename(columns={"EDFVD_test": "EDF-VD (sufficient)"})
df_edfvd_test["EDF-VD (sufficient)"] = df_edfvd_test["EDF-VD (sufficient)"].astype(bool)

In [ ]:
df_sch["schedulable"] = df_sch["is_safe"]

In [ ]:
d1 = df_sch[["taskset_position", "use_case", "schedulable", "U"]]
d2 = df_edfvd_test.melt(
    id_vars=["taskset_position", "U"], var_name="use_case", value_name="schedulable"
)

In [ ]:
df_sched_comb = pd.concat([d1, d2], ignore_index=True)

In [ ]:
# f, ax = plt.subplots(figsize=(3.5 * 46 / 98 * 2, 3.5))
f, ax = plt.subplots(figsize=(3.5, 3.5))

col_order = ["EDF-VD (sufficient)", "EDF-VD (exact)", "LWLF (exact)"]

sns.lineplot(
    data=df_sched_comb,
    x="U",
    y="schedulable",
    hue="use_case",
    ax=ax,
    markers=True,
    errorbar=None,
    style="use_case",
    ms=9,
    hue_order=col_order,
    style_order=col_order,
)

ax.set_ylabel("Schedulability ratio")
ax.set_xlabel("Average utilisation ($U^*$)")

# f.suptitle(
#     r"$P_{\mathsf{HI}} = 0.5, T^{\mathsf{min}} = 5, T^{\mathsf{max}} = 30, n=5$",
#     x=0.55,
#     # y=0.7,
# )

ax.set_title(
    r"$P_{\mathsf{HI}} = 0.5, T^{\mathsf{min}} = 5, T^{\mathsf{max}} = 30, n=5$",
)

# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles=handles, labels=labels, framealpha=1, loc="lower left")
# ax.legend(
#     handles=handles,
#     labels=labels,
#     loc="upper center",
#     ncol=2,
#     framealpha=0,
#     columnspacing=0.8,
#     bbox_to_anchor=(0.5, 1.4),
# )
sns.move_legend(
    ax,
    loc="center left",
    # ncol=2,
    framealpha=1,
    # columnspacing=0.8,
    # bbox_to_anchor=(0.5, 1.8),
    title="",
)

f.tight_layout()

In [ ]:
f.savefig(
    f"./fig_rtss/{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_scheduling.pdf",
    bbox_inches="tight",
)
f.savefig(
    "./fig_rtss/scheduling.pdf",
    bbox_inches="tight",
)